In [ ]:
!pip install torch_geometric

In [ ]:
import torch
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GCNConv
import numpy as np

In [ ]:
# Define the domain and discretization
nx, ny = 20, 20  # Grid size
x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)
xx, yy = np.meshgrid(x, y)
nodes = torch.tensor(np.vstack([xx.ravel(), yy.ravel()]).T)  # Nodes in the graph

# Define edges: connect each node to its neighbors
edges = []
for i in range(nx):
    for j in range(ny):
        if i < nx - 1:  # Horizontal edges
            edges.append([i * ny + j, (i + 1) * ny + j])
            edges.append([(i + 1) * ny + j, i * ny + j])
        if j < ny - 1:  # Vertical edges
            edges.append([i * ny + j, i * ny + (j + 1)])
            edges.append([i * ny + (j + 1), i * ny + j])

edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

# Define node features: (x, y, initial guess for u)
node_features = torch.tensor(np.c_[nodes, np.zeros((nx * ny, 1))], dtype=torch.float)

# Define the source term f(x, y) for the Poisson equation
def source_term(x, y):
    return torch.sin(torch.pi * x) * torch.sin(torch.pi * y)

source_values = torch.tensor(source_term(nodes[:, 0], nodes[:, 1]).unsqueeze(1), dtype=torch.float)

In [5]:
print(edge_index[:,1])

tensor([20,  0])


In [6]:

# Create PyTorch Geometric data object
data = Data(x=node_features, edge_index=edge_index, y=source_values)

# Define a simple GCN model
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(3, 16)
        self.conv2 = GCNConv(16, 32)
        self.conv3 = GCNConv(32, 1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = F.relu(self.conv2(x, edge_index))
        x = self.conv3(x, edge_index)
        return x

# Initialize and train the GNN
model = GCN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Training loop
model.train()
for epoch in range(1000):
    optimizer.zero_grad()
    out = model(data)
    loss = F.mse_loss(out, data.y)  # Minimize the residual of the Poisson equation
    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

# Predict the solution
model.eval()
with torch.no_grad():
    predicted_u = model(data).detach().numpy()

# Reshape the solution for visualization
predicted_u = predicted_u.reshape(nx, ny)

# Visualize the solution using matplotlib
import matplotlib.pyplot as plt

plt.imshow(predicted_u, extent=(0, 1, 0, 1), origin='lower', cmap='viridis')
plt.colorbar(label='u(x, y)')
plt.title('Predicted Solution u(x, y) for the Poisson Equation')
plt.show()